In [2]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("../staging.csv")

C:\Users\acasalro\AppData\Local\Temp\ipykernel_20836\2806958424.py:1: DtypeWarning: Columns (82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../staging.csv")


In [7]:
df["Source IP"].value_counts()

Source IP
1    670423
0      9489
Name: count, dtype: int64

In [26]:
df.columns

Index(['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Le

In [27]:
df = df.drop("Unnamed: 0", axis=1)
df = df.drop("Flow ID", axis=1)

In [28]:
def ip_col_classiffication(df, colum):
    calsif = []
    for i in range(len(df[colum])):

        nums = df[colum][i].split(".")
        nums = [int(num) for num in nums]

        if nums[0] == 172 and nums[1] >= 16 and nums[1] <= 31:
            calsif.append(1)
        elif nums[0] == 192 and nums[1] == 168:
            calsif.append(1)
        else:
            calsif.append(0)
        
    df[colum] = calsif
    return df

In [29]:
df = ip_col_classiffication(df, "Source IP")
df = ip_col_classiffication(df, "Destination IP")

In [30]:
ports = {
    "web": [80, 443, 8080],
    "netbios": [137, 138, 139, 445],
    "ssh": [22],
    "ftp": [21],
    "ldap": [389, 3268],
    "ntp": [123],
    "udp": [53],
    "kerberos": [88],
    "smtp": [465],
    "rpc": [135],
    "dns": [5353]
}

reserved = np.concatenate([p for _,p in ports.items()])

In [31]:
def ports_to_id(df, column):

    other_ports = [port for port in df[column].unique() if port not in reserved]
    
    df[column] = df[column].replace(other_ports,"Others")

    for i, (servicio, puertos) in enumerate(ports.items(), start=1):

        df[column] = df[column].replace(puertos,servicio)

    return df

In [32]:
df = ports_to_id(df, "Source Port")
df = ports_to_id(df, "Destination Port")

In [33]:
df

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,1,Others,0,web,6,2018-12-01T13:48:44.733+01:00,21513,2,2,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN
1,1,Others,0,web,6,2018-12-01T13:48:45.170+01:00,40788,2,2,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN
2,1,Others,0,Others,0,2018-12-01T13:48:45.458+01:00,6,4,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN
3,1,Others,0,web,6,2018-12-01T13:48:45.849+01:00,27959,2,2,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN
4,1,Others,0,web,6,2018-12-01T13:48:46.005+01:00,92022,2,2,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681563,1,Others,0,web,6,2018-12-01T13:48:40.924+01:00,20742,2,2,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN
681564,1,Others,0,web,6,2018-12-01T13:48:40.924+01:00,28564,2,2,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN
681565,1,Others,0,web,6,2018-12-01T13:48:40.955+01:00,28342,2,2,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN
681566,1,Others,0,Others,0,2018-12-01T13:48:41.992+01:00,4,3,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN


In [7]:
def replace_inf_and_nans_with_next_max(df, column):

    df[column] = df[column].replace([np.inf, -np.inf], np.nan)
    
    next_max = df[column].dropna().max()
    
    df[column].fillna(next_max, inplace=True)
    
    return df

In [8]:
df = replace_inf_and_nans_with_next_max(df, "Flow Bytes/s")
df = replace_inf_and_nans_with_next_max(df, "Flow Packets/s")

C:\Users\acasalro\AppData\Local\Temp\ipykernel_14316\4247702005.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(next_max, inplace=True)


In [13]:
df.to_csv('Analisys.csv', index=False)